In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import tensorflow as tf
import datetime
import struct
import time
from sklearn import linear_model
from sklearn import svm
import sklearn.model_selection
import sklearn.metrics
import sklearn.preprocessing

root_dir = "D:/Jupyter/";
logs_dir = root_dir + "Logs/"
data_dir = root_dir + 'Datasets/'

In [2]:
def mnist_read_imgs(fname):
    with open(fname, mode='rb') as f:
        (_, img_num, img_xsize, img_ysize) = struct.unpack('>IIII',f.read(4 * 4))
        data_img = np.fromfile(f, dtype=np.uint8).reshape(img_num, img_xsize, img_ysize)
    return data_img

def mnist_read_lbls(fname):
    with open(data_dir + 'MNIST/train-labels.idx1-ubyte', mode='rb') as f:
        (_, lab_num) = struct.unpack('>II', f.read(4 * 2))
        data_lab = np.fromfile(f, dtype=np.uint8)
    return data_lab


In [5]:
src_X = mnist_read_imgs(data_dir+'MNIST/train-images.idx3-ubyte')
src_y = mnist_read_lbls(data_dir+'MNIST/train-labels.idx1-ubyte')

random_seed = 42
(dev_X, test_X, dev_y, test_y) = sklearn.model_selection.train_test_split(src_X, src_y, random_state=random_seed, test_size=0.2)
(train_X, valid_X, train_y, valid_y) = sklearn.model_selection.train_test_split(dev_X, dev_y, random_state=random_seed, test_size=0.2)

In [6]:
def mnist1d_transform_imgs(x):
    return x.reshape(x.shape[0], x.shape[1] * x.shape[2]) / 255

def mnist_transform_lbls(y):
    return[1.0*(y==i) for i in range(10)]


def mnist_transform_prob(yp):
    return [(np.asarray(range(10))[(x >= np.max(x))][0], np.max(x)/np.sum(x)) for x in yp]

def mnist_confmat(y, yf):
    ty = np.asarray(y)
    tyf = np.asarray(yf)[:,0].reshape(ty.shape)
    return np.asarray([[np.sum(1*(ty==act)*(tyf==est)) for est in range(10)] for act in range(10)])

def confmat_accuracy(confmat):
    return np.sum(np.diag(confmat)) / np.sum(confmat)

def confmat_pctact(confmat):
    return confmat / np.sum(confmat, axis=0)

def confmat_accuracy_lbl(confmat):
    return np.diag(confmat) / np.sum(confmat, axis=0)

def mnist_eval_prob(y, yp):
    yf = mnist_transform_prob(yp)
    cf = mnist_confmat(y, yf)
    return confmat_accuracy(cf)

## 1D Models

In [7]:
train1d_X = mnist1d_transform_imgs(train_X)
train1d_y = mnist_transform_lbls(train_y)

valid1d_X = mnist1d_transform_imgs(valid_X)
valid1d_y = mnist_transform_lbls(valid_y)
test1d_X = mnist1d_transform_imgs(test_X)
test1d_y = mnist_transform_lbls(test_y)

In [9]:
#scikit-learn logistic regression
start = time.perf_counter()
sklrs = []
for i in range(10):
    sklr = linear_model.LogisticRegression()
    sklr.fit(train1d_X, train1d_y[i])
    sklrs = sklrs + [sklr]
print('Fitted all models:\t\t', time.perf_counter() - start)
sklr_train_yp = np.asarray([x.predict_proba(train1d_X)[:,1] for x in sklrs]).transpose()
sklr_valid_yp = np.asarray([x.predict_proba(valid1d_X)[:,1] for x in sklrs]).transpose()
print(' + calculated probabilities:\t', time.perf_counter() - start)
#~40 seconds

Fitted all models:		 40.954590970224714
 + calculated probabilities:	 41.822692847633206


In [10]:
def mnist_eval_LogisticRegression(**kargs):
    sklrs = []
    t0 = time.perf_counter()
    for i in range(10):
        sklr = linear_model.LogisticRegression(**kargs)
        sklr.fit(train1d_X, train1d_y[i])
        sklrs = sklrs + [sklr]
    t_fit = time.perf_counter() - t0
    #Calculate
    sklr_train_yp = np.asarray([x.predict_proba(train1d_X)[:,1] for x in sklrs]).transpose()
    sklr_valid_yp = np.asarray([x.predict_proba(valid1d_X)[:,1] for x in sklrs]).transpose()
    t_calc = time.perf_counter() - (t0+t_fit)
    #Evaluate
    train_res = mnist_eval_prob(train_y, sklr_train_yp)
    valid_res = mnist_eval_prob(valid_y, sklr_valid_yp)
    t_eval = time.perf_counter() - (t0+t_fit+t_calc)
    return ((train_res, valid_res), (t_fit, t_calc, t_eval), sklrs)

def mnist_eval_LinearSVM(**kargs):
    sklrs = []
    t0 = time.perf_counter()
    for i in range(10):
        sklr = svm.LinearSVC(**kargs)
        sklr.fit(train1d_X, train1d_y[i])
        sklrs = sklrs + [sklr]
    t_fit = time.perf_counter() - t0
    #Calculate
    sklr_train_yp = np.asarray([x.predict(train1d_X) for x in sklrs]).transpose()
    sklr_valid_yp = np.asarray([x.predict(valid1d_X) for x in sklrs]).transpose()
    t_calc = time.perf_counter() - (t0+t_fit)
    #Evaluate
    train_res = mnist_eval_prob(train_y, sklr_train_yp)
    valid_res = mnist_eval_prob(val_y, sklr_valid_yp)
    t_eval = time.perf_counter() - (t0+t_fit+t_calc)
    return ((train_res, valid_res), (t_fit, t_calc, t_eval), sklrs)

In [11]:
#default logistic regression
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LogisticRegression()
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)

Timing (fit / calc / eval):	 41.31127215362413 0.8263044103732113 1.0737019329884276
Accuracy (train / valid):	 0.929244791667 0.91875


In [267]:
#L1-regularization Logistic regression
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LogisticRegression(penalty='l1',C=0.4,n_jobs=4,max_iter=500)
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LogisticRegression(penalty='l1',C=0.6,n_jobs=4,max_iter=500)
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LogisticRegression(penalty='l1',C=0.8,n_jobs=4,max_iter=500)
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)

Timing (fit / calc / eval):	 22.407737832281782 0.7833809462317731 1.0519142952725815
Accuracy (train / valid):	 0.925104166667 0.920520833333
Timing (fit / calc / eval):	 25.986422583542662 0.793956220297332 1.1127934838677902
Accuracy (train / valid):	 0.926927083333 0.920104166667
Timing (fit / calc / eval):	 27.45085978850875 0.8062506182886864 1.0600432254123007
Accuracy (train / valid):	 0.92859375 0.9203125


In [268]:
#L2-regularization Logistic regression
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LogisticRegression(penalty='l2',C=0.4,n_jobs=4,max_iter=500)
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LogisticRegression(penalty='l2',C=0.6,n_jobs=4,max_iter=500)
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LogisticRegression(penalty='l2',C=0.8,n_jobs=4,max_iter=500)
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)

Timing (fit / calc / eval):	 31.344842363925636 0.7797372946060932 1.0490813089345465
Accuracy (train / valid):	 0.926354166667 0.920208333333
Timing (fit / calc / eval):	 33.28618599161382 0.7987356340054248 1.0540680143276404
Accuracy (train / valid):	 0.92796875 0.919791666667
Timing (fit / calc / eval):	 38.65521351798452 0.7909099009903002 1.064505093938351
Accuracy (train / valid):	 0.928828125 0.918854166667


In [283]:
#default linear-SVM regression
((train_acc, valid_acc), (dt_fit, dt_calc, dt_eval), _) = mnist_eval_LinearSVM()
print('Timing (fit / calc / eval):\t', dt_fit, dt_calc, dt_eval)
print('Accuracy (train / valid):\t', train_acc, valid_acc)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


Timing (fit / calc / eval):	 53.083806630464096 0.830865467320109 1.1599853800835263
Accuracy (train / valid):	 0.872708333333 0.858229166667


## Speed benchmarking: SkLearn vs NumPy vs TensorFlow

In [12]:
#Will clasify digit-9
bm_train_X = train1d_X
bm_train_y = train1d_y[9]

In [25]:
start = time.perf_counter()
sklr = linear_model.LogisticRegression().fit(bm_train_X, bm_train_y)
sklr_time = time.perf_counter() - start

In [26]:
sklr_time

4.6302586656729545

### Logistic Regression Loss & Gradient

Loss = sum(yi * ln(p(zi)) + (1 - yi) * ln(1 - p(zi)))
     = sum(yi * ln(p(zi) / (1 - p(zi)) + ln(1 - p(zi)) )

p(x) = 1 / (1 + exp(-x))
=> 1 - p(x) = 1 / (1 + exp(x))
=> p(x) / 1 - p(x) = 1 / exp(-x) = exp(x)

#### Loss = sum(yi * zi - ln(1 + exp(zi)))

dLoss / dzi = yi - 1/(1 + exp(zi)) * exp(zi)
            = yi - p(zi)
            
zi = w0 + w * x
dzi / dw0 = 1
dzi / dw  = xi

#### dLoss / dw0 = mean(yi - p(zi))
#### dLoss / dw  = mean((yi - p(zi)) * xi)

### NumPy solution

In [99]:
def logreg_calc_grad(X, y, w0, w):
    Z = w0 + np.matmul(X, w)
    dp = y.reshape(Z.shape) - 1 / (1 + np.exp(-Z))
    dw0 = np.mean(dp)
    dw = np.mean(dp * X, axis=0).reshape(w.shape)
    return (dw0, dw)

def logreg_make_step(X, y, w0, w, learning_rate=1):
    Z = w0 + np.matmul(X, w)
    dp = y.reshape(Z.shape) - 1 / (1 + np.exp(-Z))
    dw0 = np.mean(dp)
    dw = np.mean(dp * X, axis=0).reshape(w.shape)
    nw0 = w0 + learning_rate * dw0
    nw = w + learning_rate * dw
    return (nw0, nw)

In [98]:
w0 = 0
w = np.zeros((bm_train_X.shape[1], 1))
num_steps = 100
start = time.perf_counter()
for i in range(num_steps):
    (w0, w) = logreg_make_step(bm_train_X, bm_train_y, w0, w)
numpylr_time = time.perf_counter() - start

In [112]:
numpylr_Z = w0 + np.matmul(bm_train_X, w)
numpylr_p = 1 / (1 + np.exp(-numpylr_Z))
(numpylr_dw0, numpylr_dw) = logreg_calc_grad(bm_train_X, bm_train_y, w0, w)
print('dw0:', abs(numpylr_dw0))
print('dw (mean/max):', np.mean(np.abs(numpylr_dw)), np.max(np.abs(numpylr_dw)))
print('time:', numpylr_time)

dw0: 0.00636958773088
dw (mean/max): 0.000236895165698 0.00140417549946
time: 15.517000586762151


### TensorFlow All-Manual

In [168]:
tf.reset_default_graph()
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
log_dir = root_dir + 'Logs/' + dt_now

tf_w0 = tf.Variable(0.0, name='w0')
tf_w = tf.Variable(np.zeros(shape=(bm_train_X.shape[1], 1)), name='w', dtype=tf.float32)
tf_lr = tf.placeholder(name='learning_rate', dtype=tf.float32, shape=())
tf_y = tf.placeholder(name='y', dtype=tf.float32, shape=(None,1))
tf_X = tf.placeholder(name='X', dtype=tf.float32, shape=(None, 28*28))
tf_Z = tf_w0 + tf.matmul(tf_X, tf_w)
tf_p = 1 / (1 + tf.exp(-tf_Z))
tf_dp = tf_y - tf_p
tf_dw0 = tf.reduce_mean(tf_dp)
tf_dw = tf.reshape(tf.reduce_mean(tf.multiply(tf_dp, tf_X), axis=0), tf_w.shape)
tf_train_w0 = tf.assign(tf_w0, tf_w0 + tf_lr * tf_dw0)
tf_train_w = tf.assign(tf_w, tf_w + tf_lr * tf_dw)
tf_trainop = tf.group(tf_train_w, tf_train_w0)
tf_start = tf.global_variables_initializer()

tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [172]:
num_steps=100
batch = {tf_X: bm_train_X, tf_y: bm_train_y.reshape(bm_train_X.shape[0],1), tf_lr:1.0}
with tf.Session() as sess:
    sess.run(tf_start)
    start = time.perf_counter()
    for i in range(num_steps):
        sess.run(tf_trainop, feed_dict=batch)
    tfman_time = time.perf_counter() - start
    tfman_w0 = tf_w0.eval()
    tfman_w = tf_w.eval()
    tfman_p = tf_p.eval(feed_dict=batch)

### TensorFlow GradientDescent

In [221]:
tf.reset_default_graph()
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
log_dir = root_dir + 'Logs/' + dt_now

tf_w0 = tf.Variable(0.0, name='w0')
tf_w = tf.Variable(np.zeros(shape=(bm_train_X.shape[1], 1)), name='w', dtype=tf.float32)
tf_lr = tf.placeholder(name='learning_rate', dtype=tf.float32, shape=())
tf_y = tf.placeholder(name='y', dtype=tf.float32, shape=(None,1))
tf_X = tf.placeholder(name='X', dtype=tf.float32, shape=(None, 28*28))
tf_Z = tf_w0 + tf.matmul(tf_X, tf_w)
tf_p = 1 / (1 + tf.exp(-tf_Z))
tf_loss = -tf.reduce_mean(tf_y * tf_Z - tf.log(1 + tf.exp(tf_Z)))
tf_opt = tf.train.GradientDescentOptimizer(1.0)
tf_trainop = tf_opt.minimize(tf_loss)
tf_start = tf.global_variables_initializer()

tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [222]:
num_steps=100
batch = {tf_X: bm_train_X, tf_y: bm_train_y.reshape(bm_train_X.shape[0],1), tf_lr:1.0}
with tf.Session() as sess:
    sess.run(tf_start)
    start = time.perf_counter()
    for i in range(num_steps):
        sess.run(tf_trainop, feed_dict=batch)
    tfgd_time = time.perf_counter() - start
    tfgd_w0 = tf_w0.eval()
    tfgd_w = tf_w.eval()
    tfgd_p = tf_p.eval(feed_dict=batch)

In [223]:
numpylr_time, tfman_time, tfgd_time

(15.517000586762151, 67.120305629207, 13.184399634803412)

### TensorFlow NN for MNIST

In [285]:
#Equivalent of Logistic Regression
tf.reset_default_graph()
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
log_dir = root_dir + 'Logs/' + dt_now

tf_lr = tf.placeholder(name='learning_rate', dtype=tf.float32, shape=())
tf_y = tf.placeholder(name='y', dtype=tf.float32, shape=(None, 10))
tf_X = tf.placeholder(name='X', dtype=tf.float32, shape=(None, 28*28))

tf_OUT = tf.layers.dense(tf_X, 10, use_bias=True) #Linear Activation
tf_OUT_PROB = tf.nn.softmax(tf_OUT)

tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_y, logits=tf_OUT))

tf_optimizer = tf.train.GradientDescentOptimizer(learning_rate=tf_lr)

tf_init = tf.global_variables_initializer()
tf_train = tf_optimizer.minimize(tf_loss)

tffw = tf.summary.FileWriter(log_dir, tf.get_default_graph())
tf_err_summary = tf.summary.scalar('ERROR', tf_loss)

In [287]:
epoch_size=10
num_epochs=100
train_batch = {tf_X: train1d_X, tf_y: np.asarray(train1d_y).transpose(), tf_lr:1.0}
valid_batch = {tf_X: valid1d_X, tf_y: np.asarray(valid1d_y).transpose()}
with tf.Session() as sess:
    sess.run(tf_init)
    for i in range(num_epochs):
        for j in range(epoch_size):
            sess.run(tf_train, feed_dict=train_batch)
        sum_str = tf_err_summary.eval(feed_dict=valid_batch)
        valid_loss = tf_loss.eval(feed_dict=valid_batch)
        tffw.add_summary(sum_str, i)
        print(i, valid_loss)
    z = tf_OUT.eval(feed_dict=batch)
    prob = tf_OUT_PROB.eval(feed_dict=batch)

0 0.879148
1 0.588482
2 0.455498
3 0.39903
4 0.381073
5 0.369185
6 0.359958
7 0.352458
8 0.346203
9 0.340889
10 0.336308
11 0.332312
12 0.328791
13 0.325663
14 0.322862
15 0.320339
16 0.318052
17 0.31597
18 0.314064
19 0.312314
20 0.3107
21 0.309206
22 0.30782
23 0.30653
24 0.305326
25 0.304199
26 0.303143
27 0.302151
28 0.301217
29 0.300335
30 0.299503
31 0.298715
32 0.297969
33 0.29726
34 0.296587
35 0.295946
36 0.295336
37 0.294754
38 0.294198
39 0.293667
40 0.293159
41 0.292673
42 0.292207
43 0.29176
44 0.29133
45 0.290918
46 0.290522
47 0.290141
48 0.289774
49 0.28942
50 0.28908
51 0.288751
52 0.288434
53 0.288129
54 0.287833
55 0.287548
56 0.287272
57 0.287005
58 0.286747
59 0.286497
60 0.286255
61 0.286021
62 0.285794
63 0.285574
64 0.28536
65 0.285153
66 0.284952
67 0.284757
68 0.284568
69 0.284384
70 0.284205
71 0.284032
72 0.283863
73 0.283699
74 0.28354
75 0.283384
76 0.283234
77 0.283087
78 0.282944
79 0.282805
80 0.282669
81 0.282538
82 0.282409
83 0.282284
84 0.282162
85 

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("X:0", shape=(?, 784), dtype=float32) is not an element of this graph.

In [296]:
prob

array([[  4.68059257e-02,   6.15276862e-04,   1.85183878e-03, ...,
          3.08779487e-03,   1.38314292e-01,   5.08469017e-03],
       [  5.55270761e-02,   3.01036541e-03,   3.30095482e-03, ...,
          2.74824561e-03,   2.61425972e-01,   1.15526728e-02],
       [  2.83087866e-04,   8.45496178e-01,   1.47710973e-02, ...,
          1.29105533e-02,   7.19248131e-02,   1.08698476e-03],
       ..., 
       [  4.51952452e-03,   6.13750935e-01,   2.74646860e-02, ...,
          1.77786071e-02,   1.64972827e-01,   1.09976474e-02],
       [  1.14167552e-03,   2.31584981e-02,   3.56666297e-02, ...,
          5.86982677e-03,   2.36413196e-01,   1.01279598e-02],
       [  1.66436777e-01,   3.52604105e-03,   5.59435971e-03, ...,
          1.24523835e-02,   1.01005249e-01,   2.72773160e-03]], dtype=float32)

In [297]:
mnist_eval_prob(train_y, prob)

0.72562499999999996

(38400, 10)